In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8,
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)

y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



### Experiment 3: Train XGBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)

y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



### Track Experiment using MLFlow

In [9]:
models = [
    (
        "Logistic Regression",
        LogisticRegression(C=1, solver='liblinear'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [12]:
import dagshub
dagshub.init(repo_owner='amsowmya', repo_name='MLFlow_Dagshub_Demo', mlflow=True)

Accessing as amsowmya

Initialized MLflow to track repo "amsowmya/MLFlow_Dagshub_Demo"

Repository amsowmya/MLFlow_Dagshub_Demo initialized!

In [24]:
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = 'amsowmya'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '41a42ec3b792a4ae6be59d41f8caaf016dd55614'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow'

In [25]:

mlflow.set_experiment("Anomaly Detection2")
# mlflow.set_tracking_uri('https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow')

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])
        
        if 'XGB'in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2024/08/14 15:40:26 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/14 15:40:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/#/experiments/1/runs/039ba3dc57504144bc66f7f48015b8b7.
2024/08/14 15:40:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/#/experiments/1.
2024/08/14 15:40:35 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/14 15:40:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/#/experiments/1/runs/e32888b337ad43329252d2a7a2e07557.
2024/08/14 15:40:36 INFO mlflow.tracking._tracking_service.client: 🧪 View expe

In [15]:
model_name = 'XGB-smoke'
run_id = input('Please enter RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'XGB-smoke'.
2024/08/14 15:16:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-smoke, version 1
Created version '1' of model 'XGB-smoke'.
2024/08/14 15:16:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/#/experiments/1/runs/feca6a73c42249fb82c70e7c39c1f957.
2024/08/14 15:16:34 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/#/experiments/1.


In [20]:
model_version = 1
# model_uri = f"models:/{model_name}/{model_version}"
# model_uri = f"models:/XGB-smoke@test"
model_uri = 'models:/XGB-smoke/1'

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

MlflowException: The following failures occurred while downloading one or more artifacts from https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/api/2.0/mlflow-artifacts/artifacts/db1e5897210b4cfbbcbed29010d00d8e/feca6a73c42249fb82c70e7c39c1f957/artifacts/model_name:
##### File  #####
API request to https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/api/2.0/mlflow-artifacts/artifacts/db1e5897210b4cfbbcbed29010d00d8e/feca6a73c42249fb82c70e7c39c1f957/artifacts/model_name/ failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /amsowmya/MLFlow_Dagshub_Demo.mlflow/api/2.0/mlflow-artifacts/artifacts/db1e5897210b4cfbbcbed29010d00d8e/feca6a73c42249fb82c70e7c39c1f957/artifacts/model_name/ (Caused by ResponseError('too many 500 error responses'))

In [21]:
model_name = 'RandomForest'
run_id = input('Please enter RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'RandomForest'.
2024/08/14 15:33:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest, version 1
Created version '1' of model 'RandomForest'.
2024/08/14 15:33:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/#/experiments/1/runs/ce5f20cbd77841ad9d990d5721360d22.
2024/08/14 15:33:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/#/experiments/1.


In [23]:
model_version = 1
# model_uri = f"models:/{model_name}/{model_version}"
model_uri = f"models:/RandomForest@test1"

loaded_model = mlflow.sklearn.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

MlflowException: The following failures occurred while downloading one or more artifacts from https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/api/2.0/mlflow-artifacts/artifacts/db1e5897210b4cfbbcbed29010d00d8e/ce5f20cbd77841ad9d990d5721360d22/artifacts/model_name:
##### File  #####
API request to https://dagshub.com/amsowmya/MLFlow_Dagshub_Demo.mlflow/api/2.0/mlflow-artifacts/artifacts/db1e5897210b4cfbbcbed29010d00d8e/ce5f20cbd77841ad9d990d5721360d22/artifacts/model_name/ failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /amsowmya/MLFlow_Dagshub_Demo.mlflow/api/2.0/mlflow-artifacts/artifacts/db1e5897210b4cfbbcbed29010d00d8e/ce5f20cbd77841ad9d990d5721360d22/artifacts/model_name/ (Caused by ResponseError('too many 500 error responses'))